## Set the enviroments.

In [1]:
(ext:chdir "~/Development/clasp/extensions/cando/src/tests/alanine-demo/" t)

0

#### Connect to slime.

In [2]:
(start-swank)


"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 :MIPS :MIPSEL :JAVA-1.4 :JAVA-1.5 :JAVA-1.6 :JAVA-1.7)))
" 
"Read: (

T

#### Add paths.

In [3]:
*default-pathname-defaults*

#P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/"

In [6]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [7]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [8]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [9]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>
 #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [10]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [11]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [12]:
(build-hydrogens *chemagg*)

NIL

In [13]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Gather atoms to the center.

In [14]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C      S


NIL

#### Set the bounding box.

In [15]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [16]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [17]:
(cando:jostle *chemagg*)

NIL

In [18]:
(defparameter *ee* (energy:minimize *chemagg*))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000          18402.398
 minSDnP       0     2  0.000284   0.0        3025857.066          12136.674
 minSDnP       0     3  0.000236   0.0        2321337.147           5963.046
DONE absolute force test:
forceRmsMag(4195.678649).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     4  0.000000   0.0              0.000           4195.679
 minCGnP       0     5  0.000457  38.4        1541197.251           3320.063
 minCGnP       0     6  0.001254  50.6        1097840.297           2730.162
 minCGnP       0     7  0.000441  48.5        1002340.377           2173.215
 minCGnP       0     8  0.000672  71.9         867945.181           4372.728
 minCGnP       0     9  0.000171  67.9         745441.766           4212.574
 minCGnP       0    10  0.000389  80.8         652492.035 

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [19]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Assign atom types and change one of them for alanine atom type definition.

In [22]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [23]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" :assign-types nil)

atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 0 i2: 3
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 0 i2: 6
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 3 i2: 48
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 9 i2: 0
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 12 i2: 0
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 15 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 18 i2: 21
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i1: 21 i2: 30
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 21 i2: 33
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 24 i2: 18
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 27 i2: 21
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 36 i2: 39
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 39 i2: 42
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 42 i2: 18
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C/:C> i1: 45 i2: 39
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C/:C> i1: 48 i2: 39
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 51 i2: 48

0

#### Calculate each energy parameters.

In [24]:
(chem:get-energy (chem:get-stretch-component *ee*))

2.7731802863027997d0

In [25]:
(chem:get-energy (chem:get-angle-component *ee*))

1.400906566287098d0

In [26]:
(chem:get-energy (chem:get-dihedral-component *ee*))

8.724973561642626d0

In [27]:
(chem:get-energy (chem:get-nonbond-component *ee*))

10.707853642954538d0

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [28]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [29]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [30]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [31]:
(nglv::show-aggregate *solvent*)

A Jupyter Widget

#### Assign the atom types of the force field to the water box.

In [32]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [33]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(5.0 5.0 5.0)))

Total bounding box for atom centers:  18.99893108722887d0 14.183780465762425d0 13.159029044070262d0


*NEWAGG*

In [34]:
(nglv::show-aggregate *newagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [35]:
(assign-atom-types *newagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *newagg*)

NIL

In [36]:
(defparameter *e* (energy:minimize *newagg*))

======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(2.398792).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minCGnP       0     1  0.000000   0.0              0.000              2.399
 minCGnP       0     2  0.001528  31.9              9.964              1.492
 minCGnP       0     3  0.000575  36.0              9.536              0.908
 minCGnP       0     4  0.000771  37.2              9.322              0.574
DONE absolute force test:
forceRmsMag(0.396855).LT.forceTolerance(0.500000)
======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(6.377839).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     5  0.000000   0.0              0.000              6.378
 minCGnP       0     6  0.000779  27.3             13.668              3.289
 minCGnP       0     7  0.001051  52.5         

*E*

In [33]:
(leap.topology:save-amber-parm-format *newagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil)

atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 0 i2: 21
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 0 i2: 15
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 3 i2: 0
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 3 i2: 12
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 6 i2: 3
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 9 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :O/:O> i1: 21 i2: 18
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C/:C> i1: 21 i2: 24
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 27 i2: 24
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C/:C> i1: 30 i2: 24
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 33 i2: 24
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 36 i2: 33
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 39 i2: 33
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 42 i2: 33
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 45 i2: 27
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 51 i2: 27
 atom1: #<ATOM :C/:C> atom2: #<ATOM :O/:O> i1: 51 i2: 48

0

#### Calculate each energy parameters.

In [37]:
(chem:get-energy (chem:get-stretch-component *e*))

2.8061924696985225d0

In [38]:
(chem:get-energy (chem:get-angle-component *e*))

1.1616684719178318d0

In [39]:
(chem:get-energy (chem:get-dihedral-component *e*))

8.69549859900465d0

In [40]:
(chem:get-energy (chem:get-nonbond-component *e*))

-61.23827916473209d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL